In [59]:
import pandas as pd, numpy as np

In [60]:
import ortools.linear_solver.pywraplp as otlp

In [61]:
# Entrada
dados_ger = pd.read_excel('inputs_dados.xlsx', sheet_name='geracao')
dados_carga = pd.read_excel('inputs_dados.xlsx', sheet_name='carga')
dados_dep = pd.read_excel('inputs_dados.xlsx', sheet_name='dependencia')
Ng = len(dados_ger) # Número de geradores (quantas linhas possui o dataframe dados_ger)

In [62]:
# Modelo
solver = otlp.Solver('teste', otlp.Solver.GLOP_LINEAR_PROGRAMMING)

In [26]:
# Variáveis de entrada modelo para 2 dimensões
Pg = np.zeros([3,Ng]).tolist()
for t in range(3):
    for g in range(Ng):
        # Pg[g] = solver.NumVar(0, float(dados_ger.maximo[g]), 'Pg[{}]'.format([g]))
        Pg[t][g] = solver.NumVar(0, float(dados_ger.maximo[g]),f'Pg[{[t,g]}]')

In [63]:
# Variáveis de entrada para 1 dimensão
Pg = np.zeros(Ng).tolist()
for g in range(Ng):
    # Pg[g] = solver.NumVar(0, float(dados_ger.maximo[g]), 'Pg[{}]'.format([g]))
    Pg[g] = solver.NumVar(0, float(dados_ger.maximo[g]),f'Pg[{[g]}]')

In [64]:
# Restrições
solver.Add(sum([Pg[g] for g in range(Ng)])==sum(dados_carga.valor))
for c in dados_dep.carga.unique():
    solver.Add(float(dados_carga.valor[c]) <= sum([Pg[g] for g in dados_dep.gerador[dados_dep.carga==c]]))

In [65]:
# Função objetivo
solver.Minimize(sum([Pg[g]*float(dados_ger.custo[g]) for g in range(Ng)]))

In [66]:
# Resultado
results = solver.Solve()

In [67]:
# Impressão do resultado
if results == otlp.Solver.OPTIMAL:
    print('Resultado encontrado')
else:
    print("Resultado NÃO encontrado")
    
for g in range(Ng):
    print(f'Pg[{g}] = {Pg[g].solution_value()}')

Resultado encontrado
Pg[0] = 20.0
Pg[1] = 10.0
Pg[2] = 35.0
Pg[3] = 30.0
Pg[4] = 5.0


In [69]:
print(dados_ger)
print(dados_carga)

   id  maximo  custo
0   0      20   0.10
1   1      10   0.05
2   2      40   0.30
3   3      50   0.40
4   4       5   0.01
   id  valor
0   0     50
1   1     20
2   2     30
